In [13]:
#installs
!pip install pandas
!pip install db-sqlite3


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
#imports
import pandas as pd
import re
from collections import defaultdict
import json
import sqlite3
import os
import shutil

In [2]:
# Set of predefined common words to be ignored. The list can be modified as required. For the purpose of this program, these stopwords are as provided by the NLTK library

stop_words={'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"}

In [23]:
file_name='input.csv' # file_name to be processed, should be present in the "input" directory
source_path = 'input/'+file_name
input_df = pd.read_csv(source_path) # reading the CSV file into pandas dataframe

In [17]:
display(input_df)

,id,source,original_text
0,1001234,OnlineSource1,It is not the critic who counts; not the one w...
1,1001235,OnlineSource2,"A scrupulous writer, in every sentence that he..."
2,1001236,OnlineSource2,The happiness of your life depends upon the qu...


In [5]:
input_df["word_frequency"]='' # creating an empty column in the dataframe to hold word frequency

In [7]:
display(input_df)

,id,source,original_text,word_frequency
0,1001234,OnlineSource1,It is not the critic who counts; not the one w...,
1,1001235,OnlineSource2,"A scrupulous writer, in every sentence that he...",
2,1001236,OnlineSource2,The happiness of your life depends upon the qu...,


In [6]:
pattern = r"[-+]?\d*\.\d+|\d+|[^\w\s]" # regex to match unwanted characters - integers, floating values, punctuations and emojis

In [8]:
# this piece of code will loop over all the rows in the dataframe and get the word frequency
for ind in input_df.index:
    
    text=input_df['original_text'][ind]
    text = re.sub(pattern, '', text).strip() # removing unwanted characters as defined by earlier regex
    
    word_freq_dict = defaultdict(int) # dictionary to hold word frequency per row

    for word in text.casefold().split():
        
        if word not in stop_words and word != '': # condition to ignore the stopwords and empty strings
            word_freq_dict[word] += 1
            
    serialised = json.dumps(word_freq_dict) # converting ditionary to a JSON object to be further stored in Database
    input_df.loc[ind,'word_frequency']= serialised # Updating the word_frequency column in dataframe

In [9]:
display(input_df)

,id,source,original_text,word_frequency
0,1001234,OnlineSource1,It is not the critic who counts; not the one w...,"{""critic"": 1, ""counts"": 1, ""one"": 2, ""points"":..."
1,1001235,OnlineSource2,"A scrupulous writer, in every sentence that he...","{""scrupulous"": 1, ""writer"": 1, ""every"": 1, ""se..."
2,1001236,OnlineSource2,The happiness of your life depends upon the qu...,"{""happiness"": 1, ""life"": 3, ""depends"": 1, ""upo..."


In [10]:
input_df.to_csv('output.csv', index=False) # saving resultant dataframe as CSV

In [11]:
# Database - SQLite for disk-based database solution
conn = sqlite3.connect('Word_Frequency.db')

input_df.to_sql('Word_Frequency', conn, if_exists='replace', index=False) # converted the dataframe to SQLite DB table
cur = conn.cursor()

In [12]:
# SQL query to retrieve data from the database 
cur.execute("SELECT * FROM Word_Frequency") 

result = cur.fetchall() # the method fetches all rows of a query result set and returns a list of tuples

In [13]:
# displaying rows returned
for row in result: 
    print(row) 
    print("\n")

(1001234, 'OnlineSource1', 'It is not the critic who counts; not the one who points out how the strong person stumbles, or where the doer of deeds could have done them better. The credit belongs to the one who is actually in the arena, whose face is marred by dust and sweat and blood; who strives valiantly; who errs, who comes short again and again, because there is no effort without error and shortcoming; but who does actually strive to do the deeds; who knows great enthusiasms, the great devotions; who spends their self in a worthy cause; who at the best knows in the end the triumph of high achievement, and who at the worst, if they fail, at least fail while daring greatly, so that their place shall never be with those cold and timid souls who neither know victory nor defeat', '{"critic": 1, "counts": 1, "one": 2, "points": 1, "strong": 1, "person": 1, "stumbles": 1, "doer": 1, "deeds": 2, "could": 1, "done": 1, "better": 1, "credit": 1, "belongs": 1, "actually": 2, "arena": 1, "whos

In [14]:
os.mkdir('processed') # creating a new directory to move the processed file to

In [26]:
destination_path = 'processed/'+file_name
shutil.move(source_path, destination_path)

'processed/input.csv'